In [1]:
# GPU kontrolü
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
Memory: 14.74 GB


## Kurulum

Sadece bir kez çalıştır:

In [2]:
# Colab'da gerekli kütüphaneleri yükle
!pip install -q transformers datasets torch scikit-learn tqdm

print("✓ Kütüphaneler yüklendi!")

✓ Kütüphaneler yüklendi!


In [3]:
# GitHub'dan projeyi klonla
!rm -rf Mitre_Attack_TTP_Mapping
!git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git
%cd Mitre_Attack_TTP_Mapping

print("✓ Proje dosyaları yüklendi!")

Cloning into 'Mitre_Attack_TTP_Mapping'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 1), reused 27 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 26.58 KiB | 13.29 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/Mitre_Attack_TTP_Mapping
✓ Proje dosyaları yüklendi!
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 27 (delta 1), reused 27 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 26.58 KiB | 13.29 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/Mitre_Attack_TTP_Mapping
✓ Proje dosyaları yüklendi!


## Eğitim Başlat 🚀

Aşağıdaki hücrelerden birini çalıştır:

In [4]:
# BERT-base ile eğitim (3 epoch, batch 16)
!python main.py \
    --model bert-base-uncased \
    --epochs 3 \
    --batch_size 16 \
    --learning_rate 2e-5 \
    --device cuda


CTI-BERT TTP TAGGING - TRAINING PIPELINE

Configuration:
  Model: bert-base-uncased
  Epochs: 3
  Batch size: 16
  Learning rate: 2e-05
  Max length: 512
  Device: cuda


STEP 1: DATA PREPARATION

Loading dataset: tumeteor/Security-TTP-Mapping
README.md: 4.02kB [00:00, 21.5MB/s]
README.md: 4.02kB [00:00, 21.5MB/s]
derived_procedure_train.tsv: 1.13MB [00:00, 49.5MB/s]
expert_train.tsv: 128kB [00:00, 135MB/s]
derived_procedure_train.tsv: 1.13MB [00:00, 49.5MB/s]
expert_train.tsv: 128kB [00:00, 135MB/s]
procedure_train.tsv: 874kB [00:00, 160MB/s]
procedure_train.tsv: 874kB [00:00, 160MB/s]
tram_train.tsv: 546kB [00:00, 142MB/s]
tram_train.tsv: 546kB [00:00, 142MB/s]
derived_procedure_dev.tsv: 215kB [00:00, 108MB/s]
derived_procedure_dev.tsv: 215kB [00:00, 108MB/s]
expert_dev.tsv: 19.0kB [00:00, 49.8MB/s]
procedure_dev.tsv: 153kB [00:00, 83.2MB/s]
expert_dev.tsv: 19.0kB [00:00, 49.8MB/s]
procedure_dev.tsv: 153kB [00:00, 83.2MB/s]
tram_dev.tsv: 97.4kB [00:00, 86.2MB/s]
tram_dev.tsv: 97.4kB

In [5]:
# SecBERT (security domain model) ile eğitim
!python main.py \
    --model jackaduma/SecBERT \
    --epochs 3 \
    --batch_size 16 \
    --device cuda


CTI-BERT TTP TAGGING - TRAINING PIPELINE

Configuration:
  Model: jackaduma/SecBERT
  Epochs: 3
  Batch size: 16
  Learning rate: 2e-05
  Max length: 512
  Device: cuda


STEP 1: DATA PREPARATION

Loading dataset: tumeteor/Security-TTP-Mapping
Available splits: ['train', 'validation', 'test']
Available splits: ['train', 'validation', 'test']
Using validation split as test set
Train samples: 14936
Test samples: 2630
Total unique labels (MITRE ATT&CK Techniques): 499
Using validation split as test set
Train samples: 14936
Test samples: 2630
Total unique labels (MITRE ATT&CK Techniques): 499
Loading tokenizer: jackaduma/SecBERT
Loading tokenizer: jackaduma/SecBERT
config.json: 100% 467/467 [00:00<00:00, 2.00MB/s]
config.json: 100% 467/467 [00:00<00:00, 2.00MB/s]
vocab.txt: 378kB [00:00, 17.6MB/s]
vocab.txt: 378kB [00:00, 17.6MB/s]
Tokenizing texts...
Tokenizing texts...
Data preparation complete!

Label list saved to: ./outputs/jackaduma_SecBERT_20251130_114706/labels.json

STEP 2: MODEL

In [12]:
# Sadece metrikleri ve son modeli zip'le (checkpoint'ler hariç)
from google.colab import files
import shutil
import os

# Geçici klasör oluştur
os.makedirs('/content/results_small', exist_ok=True)

# Her model klasöründen sadece önemli dosyaları kopyala
for model_dir in os.listdir('./outputs'):
    src = f'./outputs/{model_dir}'
    dst = f'/content/results_small/{model_dir}'
    os.makedirs(dst, exist_ok=True)
    
    # Sadece bu dosyaları kopyala
    for file in ['labels.json', 'training_history.json', 'evaluation_metrics.json', 'summary.json']:
        src_file = f'{src}/{file}'
        if os.path.exists(src_file):
            shutil.copy(src_file, f'{dst}/{file}')

# Zip oluştur (checkpoint'ler olmadan - ~1MB)
shutil.make_archive('/content/results_metrics', 'zip', '/content/results_small')
print(f"✓ Metrics zip oluşturuldu: {os.path.getsize('/content/results_metrics.zip') / 1024 / 1024:.2f} MB")
files.download('/content/results_metrics.zip')

✓ Metrics zip oluşturuldu: 0.01 MB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
# Son model + metrikler (~1-2GB)
from google.colab import files
import shutil
import os

os.makedirs('/content/results_final', exist_ok=True)

for model_dir in os.listdir('./outputs'):
    src = f'./outputs/{model_dir}'
    dst = f'/content/results_final/{model_dir}'
    os.makedirs(dst, exist_ok=True)
    
    # Sadece final_model.pt ve metrikler
    for file in ['final_model.pt', 'labels.json', 'training_history.json', 
                 'evaluation_metrics.json', 'summary.json']:
        src_file = f'{src}/{file}'
        if os.path.exists(src_file):
            shutil.copy(src_file, f'{dst}/{file}')

shutil.make_archive('/content/results_final', 'zip', '/content/results_final')
size_mb = os.path.getsize('/content/results_final.zip') / 1024 / 1024
print(f"✓ Final model zip: {size_mb:.2f} MB")
files.download('/content/results_final.zip')

✓ Final model zip: 685.11 MB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## Manuel İndirme Talimatları 📥

**Colab'da dosyaları bulmak için:**

1. Sol taraftaki **📁 Files** ikonuna tıkla
2. `/content/results_metrics.zip` dosyasını bul
3. Dosyaya **sağ tıkla** > **Download** seç

**Veya terminalde görüntüle:**

ValueError: mount failed

In [ ]:
# Drive'ı mount et ve kopyala
from google.colab import drive
import os

# Mount et
if not os.path.exists('/content/drive/MyDrive'):
    print("Drive mount ediliyor...")
    drive.mount('/content/drive', force_remount=True)
else:
    print("Drive zaten mount edilmiş")

# Kopyala
!cp /content/results_metrics.zip /content/drive/MyDrive/

# Kontrol et
!ls -lh /content/drive/MyDrive/results_metrics.zip

print("✅ Google Drive > MyDrive/results_metrics.zip kaydedildi!")

cp: cannot create regular file '/content/drive/MyDrive/': No such file or directory
ls: cannot access '/content/drive/MyDrive/results_metrics.zip': No such file or directory
✅ Başarılı! Google Drive > MyDrive klasöründe 'results_metrics.zip' dosyasını bul
ls: cannot access '/content/drive/MyDrive/results_metrics.zip': No such file or directory
✅ Başarılı! Google Drive > MyDrive klasöründe 'results_metrics.zip' dosyasını bul


## Google Drive'a Kaydet 💾

## Sonuçları İndir 📥

In [7]:
!ls -lh /content/results.zip

-rw-r--r-- 1 root root 5.8G Nov 30 12:49 /content/results.zip
